In [0]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf

import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras import regularizers
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2

from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras.applications.inception_v3 import preprocess_input

import cv2
import os
import random
import collections
from collections import defaultdict

from shutil import copy
from shutil import copytree, rmtree

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.image as img
%matplotlib inline

In [0]:
n_classes = 50                # Setting Parameters 

epochs = 15
nb_train_samples =  1500
nb_validation_samples = 1389



In [4]:
# Traning only for 50 classes 

'''. Folders spilted as :-    First 50 ----> 1. Train.  ---> class 1
                                                        ---> Class 2
                                                        .....class 50
                                       ----> 2. Test.   ---> class 1
                                                        ---> Class 2
                                                        .....class 50 '''
                                    


img_width, img_height = 299, 299
train_data_dir = '/content/drive/My Drive/ML codes/Bird Detection/Dataset/CUB_200_2011/First 50/train'
validation_data_dir = '/content/drive/My Drive/ML codes/Bird Detection/Dataset/CUB_200_2011/First 50/test'
batch_size = 16


                    # Form keras using imgdatagenrator for managing data


train_datagen = ImageDataGenerator(
      preprocessing_function=preprocess_input,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
      train_data_dir,
      target_size=(img_height, img_width),
      batch_size=batch_size,
      class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
      validation_data_dir,
      target_size=(img_height, img_width),
      batch_size=batch_size,
      class_mode='categorical')


Found 1500 images belonging to 50 classes.
Found 1389 images belonging to 50 classes.


In [7]:
# Constucting MODEL

model_nm = 'DenseNet169'+'__'            ##   Enter model nm 
path='/content/drive/My Drive/ML/Bird_classification_model/'

bestmodel_path = path+'bestmodel_'+model_nm+str(n_classes)+'class.hdf5'
trainedmodel_path = path+'trainedmodel_'+model_nm+str(n_classes)+'class.hdf5'
history_path = path+'history_'+model_nm+str(n_classes)+'.log'



inception = tf.keras.applications.DenseNet169(weights='imagenet', include_top=False)

x = inception.output
x = GlobalAveragePooling2D()(x)
x = Dense(128,activation='relu')(x)
x = Dropout(0.2)(x)

predictions = Dense(n_classes,kernel_regularizer=regularizers.l2(0.005), activation='softmax')(x)
                                                                                                                  #.  Compiling model.
model = Model(inputs=inception.input, outputs=predictions)
model.compile(optimizer=SGD(lr=0.001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])


checkpoint = ModelCheckpoint(filepath=bestmodel_path, verbose=1, save_best_only=True)
csv_logger = CSVLogger(history_path)

class_map = train_generator.class_indices


51879936/51877672 [==============================] - 3s 0us/step


In [0]:
#.  Fitting model
history = model.fit_generator(train_generator,
                      steps_per_epoch = nb_train_samples // batch_size,
                      validation_data=validation_generator,
                      validation_steps=nb_validation_samples // batch_size,
                      epochs=epochs,
                      verbose=1,
                      callbacks=[csv_logger, checkpoint])


Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/15
93/93 [==============================] - ETA: 0s - loss: 3.9900 - accuracy: 0.1193
Epoch 00001: val_loss improved from inf to 3.30898, saving model to /content/drive/My Drive/ML/Bird_classification_model/bestmodel_DenseNet169__50class.hdf5
93/93 [==============================] - 784s 8s/step - loss: 3.9900 - accuracy: 0.1193 - val_loss: 3.3090 - val_accuracy: 0.3285
Epoch 2/15
93/93 [==============================] - ETA: 0s - loss: 2.7297 - accuracy: 0.4434
Epoch 00002: val_loss improved from 3.30898 to 1.93482, saving model to /content/drive/My Drive/ML/Bird_classification_model/bestmodel_DenseNet169__50class.hdf5
93/93 [==============================] - 59s 632ms/step - loss: 2.7297 - accuracy: 0.4434 - val_loss: 1.9348 - val_accuracy: 0.6076
Epoch 3/15
93/93 [==============================] - ETA: 0s - loss: 1.7669 - accuracy: 0.6752
Epoch 00003: val_loss improved from 1.93482 to 1.44056, saving

In [0]:
# Saving model

model.save('/content/drive/My Drive/ML/Bird_classification_model/trainedmodel_Inception__50class.hdf5')
class_map = train_generator.class_indices

In [0]:
# Loading Model
import tensorflow as tf
ms=tf.keras.models.load_model(
    '/content/drive/My Drive/ML/Bird_classification_model/trainedmodel_Inception__50class.hdf5', compile=True)

In [0]:
# Evaluation model

p=ms.evaluate_generator(generator=validation_generator)
path='/content/drive/My Drive/ML/'
with open(path+'All_Performance.txt','a') as f:
  f.write('\n')
  f.write(str(model_nm+"-->"+str(p))+'  No Of Epoch = '+str(epochs)+'   No Of Classes = '+str(n_classes))

In [0]:
# Predicting Samples


def predict_class(model, images, show = True):
    img = image.load_img(images, target_size=(299, 299))
    img = image.img_to_array(img)                    
    img = np.expand_dims(img, axis=0)         
    img = preprocess_input(img)                                      

    pred = ms.predict(img)
    
    index = np.argmax(pred)
    l=list(validation_generator.class_indices)   
    pred_value = l[index]
    #print(pred)
    if show:
        plt.imshow(img[0])                           
        plt.axis('off')
        print("Predicted class  = ",pred_value)
        plt.title(pred_value)
        plt.show()

# Give path of image to predict
path_2='/content/drive/My Drive/ML codes/Bird Detection/Dataset/CUB_200_2011/My_data/train/002.Laysan_Albatross/Laysan_Albatross_0083_756.jpg'
  
predict_class(model,path_2,show = True)

In [0]:
# Confusion matrix
from sklearn import metrics
import numpy as np
predictions = ms.predict_generator(validation_generator)

In [0]:
predicted_classes = np.argmax(predictions, axis=1)
print(len(predicted_classes))

true_classes = validation_generator.classes
class_labels = list(validation_generator.class_indices.keys())   

print(len(class_labels))
print(len(true_classes))
report = metrics.classification_report(true_classes, predicted_classes, target_names=class_labels)
#print(report)    
confusion_matrix = metrics.confusion_matrix(y_true=true_classes, y_pred=predicted_classes)  # shape=(12, 12)
confusion_matrix

'''import numpy as np

def plot_confusion_matrix_2(cm,
                      target_names,
                      title='Confusion matrix',
                      cmap=None,
                      normalize=True):
  FONT_SIZE = 8
  accuracy = np.trace(cm) / float(np.sum(cm))
  misclass = 1 - accuracy
  if cmap is None:
    cmap = plt.get_cmap('Blues')

  plt.figure(figsize=(8*2, 6*2))    # 8, 6
  plt.imshow(cm, interpolation='nearest', cmap=cmap)
  plt.title(title)
  plt.colorbar()

  if target_names is not None:
    tick_marks = np.arange(len(target_names))
    plt.xticks(tick_marks, target_names, rotation=90, fontsize=FONT_SIZE)
    plt.yticks(tick_marks, target_names, fontsize=FONT_SIZE)

  if normalize:
     cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


  thresh = cm.max() / 1.5 if normalize else cm.max() / 2
  for i in range (cm.shape[0]):
    for j in range (cm.shape[1]):
      if normalize:
        plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                 horizontalalignment="center",
                 fontsize=FONT_SIZE,
                 color="white" if cm[i, j] > thresh else "black")
      else:
        plt.text(j, i, "{:,}".format(cm[i, j]),
                 horizontalalignment="center",
                 fontsize=FONT_SIZE,
                 color="white" if cm[i, j] > thresh else "black")
  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
  from matplotlib.pyplot import figure
  plt.figure(figsize=(800, 600))
  plt.savefig('confusion_matrix.png', format="png")
  #print(plt.savefig('content/confusion_matrix.png'))
  plt.show()
  plt.close()

  


plot_confusion_matrix_2(confusion_matrix,class_labels,normalize=True)'''

In [0]:
confusion_matrix.diagonal()

In [0]:
# Analysing model


import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()